In [ ]:
import numpy as np

try:
    import plotly.graph_objects as go
    HAS_PLOTLY = True
except ImportError:
    HAS_PLOTLY = False


##############################################################################
#                          1. Data Structures
##############################################################################

class Material:
    """
    Stores material properties:
      E: Young's Modulus
      G: Shear Modulus
    """
    def __init__(self, E, G, name="Material"):
        self.E = E
        self.G = G
        self.name = name


class Section:
    """
    Cross-section properties (functions of x, if non-prismatic).

    For prismatic: A_func, Iy_func, Iz_func, J_func are constant-return lambdas.
    shear_factor_y, shear_factor_z -> Timoshenko shear area or correction factor.
    """
    def __init__(
        self,
        A_func,
        Iy_func,
        Iz_func,
        J_func,
        is_prismatic=True,
        name="Section",
        shear_factor_y=None,
        shear_factor_z=None
    ):
        self.A_func = A_func
        self.Iy_func = Iy_func
        self.Iz_func = Iz_func
        self.J_func  = J_func
        self.is_prismatic = is_prismatic
        self.name = name
        self.shear_factor_y = shear_factor_y
        self.shear_factor_z = shear_factor_z


class Element:
    """
    ni, nj: node indices (1-based).
    mat_id: Material ID
    sec_id: Section ID
    beam_theory: "EB" or "Timoshenko"
    """
    def __init__(
        self,
        ni,
        nj,
        mat_id,
        sec_id,
        beam_theory="EB",
        name="Element"
    ):
        self.ni = ni
        self.nj = nj
        self.mat_id = mat_id
        self.sec_id = sec_id
        self.beam_theory = beam_theory
        self.name = name


##############################################################################
#                          2. Model3DFrame Class
##############################################################################

class Model3DFrame:
    """
    Main class to define and solve a 3D frame model:
      - Nodes
      - Materials & Sections
      - Elements
      - Boundary Conditions
      - Nodal Loads
    Supports:
      - Euler–Bernoulli or Timoshenko beams
      - Non-prismatic sections (subdivided)
      - Linear or basic geometric nonlinear analysis
    """

    def __init__(self, name="My3DFrame"):
        self.name = name
        self.nodes = []             # list of [x,y,z]
        self.materials = {}         # {mat_id: Material}
        self.sections = {}          # {sec_id: Section}
        self.elements = []          # list of Element
        self.restrained_dofs = []   # store BC as a list of DOF indices (1-based)
        self.nodal_forces = None    # array of size n_nodes*6
        self.nprism_segments = 1    # for non-prismatic
        self.use_nonlinear = False  # toggle linear vs. nonlinear

    # ---------------------------------------------------------------------
    #                         Node & Element Definition
    # ---------------------------------------------------------------------
    def add_node(self, x, y, z):
        """Adds a node and returns node ID (1-based)."""
        self.nodes.append([x,y,z])
        return len(self.nodes)

    def add_material(self, mat_id, E, G, name=""):
        """Add or define a material in the dictionary."""
        self.materials[mat_id] = Material(E, G, name)

    def add_section(
        self,
        sec_id,
        A_func,
        Iy_func,
        Iz_func,
        J_func,
        is_prismatic=True,
        name="",
        shear_factor_y=None,
        shear_factor_z=None
    ):
        """Add or define a section in the dictionary."""
        self.sections[sec_id] = Section(
            A_func,
            Iy_func,
            Iz_func,
            J_func,
            is_prismatic,
            name,
            shear_factor_y,
            shear_factor_z
        )

    def add_element(self, ni, nj, mat_id, sec_id, beam_theory="EB", name=""):
        """Add an Element to the model."""
        self.elements.append(
            Element(ni, nj, mat_id, sec_id, beam_theory, name)
        )

    # ---------------------------------------------------------------------
    #                          Boundary Conditions
    # ---------------------------------------------------------------------
    def fix_node(self, node_id):
        """
        Fix all 6 DOFs at a node. 
        node_id is 1-based.
        """
        start_dof = (node_id-1)*6 + 1
        self.restrained_dofs.extend(range(start_dof, start_dof+6))

    def restrain_dof(self, node_id, dof_id):
        """
        Restrains a single DOF at node_id (1-based).
        dof_id in [1..6]: 1->ux,2->uy,3->uz,4->rx,5->ry,6->rz
        """
        global_dof = (node_id-1)*6 + dof_id
        self.restrained_dofs.append(global_dof)

    # ---------------------------------------------------------------------
    #                               Loads
    # ---------------------------------------------------------------------
    def initialize_nodal_forces(self):
        """Initialize nodal force array with zeros."""
        n = len(self.nodes)
        self.nodal_forces = np.zeros(n*6)

    def apply_nodal_load(self, node_id, dof_id, magnitude):
        """
        node_id is 1-based, dof_id in [1..6], magnitude is float.
        E.g.: -10000 for downward in z.
        """
        if self.nodal_forces is None:
            self.initialize_nodal_forces()
        index = (node_id-1)*6 + (dof_id-1)
        self.nodal_forces[index] += magnitude

    # Placeholder for uniform distributed loads
    def apply_distributed_load(self, elem_id, w, local_axis='y'):
        """
        *Optional* – could implement a fixed-end approach, then transform to global 
        nodal forces. For demonstration, left unimplemented.
        """
        pass

    # ---------------------------------------------------------------------
    #                         RUN ANALYSIS
    # ---------------------------------------------------------------------
    def run_analysis(self):
        """
        1) Build global stiffness
        2) Apply BC
        3) Solve (linear or nonlinear)
        4) Compute reactions
        5) Print displacements & reactions
        """
        # Convert node list to array
        node_array = np.array(self.nodes)
        ndof = len(node_array)*6

        # If no nodal_forces defined, initialize
        if self.nodal_forces is None:
            self.initialize_nodal_forces()
        F_global = self.nodal_forces.copy()

        # 1) Build global stiffness (linear assembly)
        K_global, elem_dof_maps = assemble_linear_system(
            node_array,
            self.elements,
            self.materials,
            self.sections,
            nprism_segments=self.nprism_segments
        )

        # 2) Apply BC & solve
        if not self.use_nonlinear:
            # Linear
            K_red, F_red = apply_boundary_conditions(K_global, F_global, self.restrained_dofs)
            U_red = solve_linear_system(K_red, F_red)
            U_full = reconstruct_full_displacements(U_red, ndof, self.restrained_dofs)
        else:
            # Nonlinear
            U_init = np.zeros(ndof)
            U_full = solve_nonlinear_newton_raphson(
                U_init, F_global,
                node_array,
                self.elements,
                self.materials,
                self.sections,
                elem_dof_maps,
                self.restrained_dofs
            )

        # 3) Reactions
        R_all = K_global @ U_full - F_global
        reactions = {}
        for dof_id in self.restrained_dofs:
            reactions[dof_id] = R_all[dof_id-1]

        # Print results
        print("\n=== Displacements ===")
        for i_node in range(len(self.nodes)):
            ux = U_full[i_node*6+0]
            uy = U_full[i_node*6+1]
            uz = U_full[i_node*6+2]
            rx = U_full[i_node*6+3]
            ry = U_full[i_node*6+4]
            rz = U_full[i_node*6+5]
            print(f"Node {i_node+1} => Ux={ux:.6e}, Uy={uy:.6e}, Uz={uz:.6e}, "
                  f"Rx={rx:.6e}, Ry={ry:.6e}, Rz={rz:.6e}")

        print("\n=== Reactions at Restrained DOFs ===")
        for dof_id in self.restrained_dofs:
            print(f" DOF {dof_id}: {reactions[dof_id]:.2f}")

        return U_full, reactions, elem_dof_maps, K_global


##############################################################################
#               3. Solver / Stiffness Routines (Same File)
##############################################################################

def build_transformation_matrix_3d(coord_i, coord_j):
    xi, yi, zi = coord_i
    xj, yj, zj = coord_j
    dx, dy, dz = xj - xi, yj - yi, zj - zi
    L = np.sqrt(dx*dx + dy*dy + dz*dz)
    if L < 1e-12:
        raise ValueError("Zero-length element")

    lx = dx/L
    ly = dy/L
    lz = dz/L

    # reference vector
    ref = np.array([0,0,1], dtype=float)
    if abs(lx) < 1e-8 and abs(ly) < 1e-8:
        ref = np.array([0,1,0], dtype=float)

    x_local = np.array([lx, ly, lz])
    y_local = np.cross(x_local, ref)
    n_y = np.linalg.norm(y_local)
    if n_y < 1e-12:
        ref = np.array([0,1,0], dtype=float)
        y_local = np.cross(x_local, ref)
        n_y = np.linalg.norm(y_local)
    y_local /= n_y
    z_local = np.cross(x_local, y_local)

    R = np.array([
        [x_local[0], x_local[1], x_local[2]],
        [y_local[0], y_local[1], y_local[2]],
        [z_local[0], z_local[1], z_local[2]],
    ])

    T = np.zeros((12,12))
    T[0:3,0:3]   = R
    T[3:6,3:6]   = R
    T[6:9,6:9]   = R
    T[9:12,9:12] = R

    return T, L

def local_stiffness_3d_euler_bernoulli(E, G, A, Iy, Iz, J, L):
    """3D Euler-Bernoulli local stiffness (12x12), ignoring shear deformation."""
    K = np.zeros((12,12))

    # Axial
    k_ax = E*A / L
    K[0,0] =  k_ax
    K[0,6] = -k_ax
    K[6,0] = -k_ax
    K[6,6] =  k_ax

    # Torsion
    k_tors = G*J / L
    K[3,3]   =  k_tors
    K[3,9]   = -k_tors
    K[9,3]   = -k_tors
    K[9,9]   =  k_tors

    # Bending about z => deflection in y
    EIz = E*Iz
    bz12 = 12.0*EIz/(L**3)
    bz6  =  6.0*EIz/(L**2)
    bz4  =  4.0*EIz/L
    bz2  =  2.0*EIz/L

    # Node i => (uy=1, rz=5), Node j => (uy=7, rz=11)
    K[1,1]    =  bz12
    K[1,5]    =  bz6
    K[1,7]    = -bz12
    K[1,11]   =  bz6
    K[5,1]    =  bz6
    K[5,5]    =  bz4
    K[5,7]    = -bz6
    K[5,11]   =  bz2
    K[7,1]    = -bz12
    K[7,5]    = -bz6
    K[7,7]    =  bz12
    K[7,11]   = -bz6
    K[11,1]   =  bz6
    K[11,5]   =  bz2
    K[11,7]   = -bz6
    K[11,11]  =  bz4

    # Bending about y => deflection in z
    EIy = E*Iy
    by12 = 12.0*EIy/(L**3)
    by6  =  6.0*EIy/(L**2)
    by4  =  4.0*EIy/L
    by2  =  2.0*EIy/L

    K[2,2]    =  by12
    K[2,4]    = -by6
    K[2,8]    = -by12
    K[2,10]   = -by6
    K[4,2]    = -by6
    K[4,4]    =  by4
    K[4,8]    =  by6
    K[4,10]   =  by2
    K[8,2]    = -by12
    K[8,4]    =  by6
    K[8,8]    =  by12
    K[8,10]   =  by6
    K[10,2]   = -by6
    K[10,4]   =  by2
    K[10,8]   =  by6
    K[10,10]  =  by4

    return K

def local_stiffness_3d_timoshenko(E, G, A, Iy, Iz, J, L,
                                  kappa=1.0,
                                  shear_area_y=None,
                                  shear_area_z=None):
    """3D Timoshenko local stiffness (12x12), includes shear deformation."""
    if shear_area_y is None:
        shear_area_y = kappa*A
    if shear_area_z is None:
        shear_area_z = kappa*A

    K = np.zeros((12,12))

    # Axial
    k_ax = E*A / L
    K[0,0]   =  k_ax
    K[0,6]   = -k_ax
    K[6,0]   = -k_ax
    K[6,6]   =  k_ax

    # Torsion
    k_tors = G*J / L
    K[3,3]   =  k_tors
    K[3,9]   = -k_tors
    K[9,3]   = -k_tors
    K[9,9]   =  k_tors

    # Shear factor for bending about z => deflection y
    # psi_z
    psi_z = 1.0 / (1.0 + (12*E*Iz)/(shear_area_z*G*L**2))
    EIz = E*Iz
    bz12 = 12*EIz*psi_z / (L**3)
    bz6  =  6*EIz*psi_z / (L**2)
    bz4  =  4*EIz*psi_z / L
    bz2  =  2*EIz*psi_z / L

    # fill sub-block
    K[1,1]    =  bz12
    K[1,5]    =  bz6
    K[1,7]    = -bz12
    K[1,11]   =  bz6
    K[5,1]    =  bz6
    K[5,5]    =  bz4
    K[5,7]    = -bz6
    K[5,11]   =  bz2
    K[7,1]    = -bz12
    K[7,5]    = -bz6
    K[7,7]    =  bz12
    K[7,11]   = -bz6
    K[11,1]   =  bz6
    K[11,5]   =  bz2
    K[11,7]   = -bz6
    K[11,11]  =  bz4

    # Shear factor for bending about y => deflection z
    psi_y = 1.0 / (1.0 + (12*E*Iy)/(shear_area_y*G*L**2))
    EIy = E*Iy
    by12 = 12*EIy*psi_y/(L**3)
    by6  =  6*EIy*psi_y/(L**2)
    by4  =  4*EIy*psi_y / L
    by2  =  2*EIy*psi_y / L

    K[2,2]    =  by12
    K[2,4]    = -by6
    K[2,8]    = -by12
    K[2,10]   = -by6
    K[4,2]    = -by6
    K[4,4]    =  by4
    K[4,8]    =  by6
    K[4,10]   =  by2
    K[8,2]    = -by12
    K[8,4]    =  by6
    K[8,8]    =  by12
    K[8,10]   =  by6
    K[10,2]   = -by6
    K[10,4]   =  by2
    K[10,8]   =  by6
    K[10,10]  =  by4

    return K

def integrate_nonprismatic_stiffness(L, n_segments, mat, sec, beam_theory="EB"):
    """
    Subdivide element into n_segments => sum local sub-stiffnesses for approximation.
    """
    dx = L / n_segments
    K_equiv = np.zeros((12,12))
    E = mat.E
    G = mat.G

    for i_seg in range(n_segments):
        x_mid = (i_seg+0.5)*dx
        A  = sec.A_func(x_mid)
        Iy = sec.Iy_func(x_mid)
        Iz = sec.Iz_func(x_mid)
        J  = sec.J_func(x_mid)

        if beam_theory=="EB":
            K_sub = local_stiffness_3d_euler_bernoulli(E, G, A, Iy, Iz, J, dx)
        else:
            sfy = sec.shear_factor_y or A
            sfz = sec.shear_factor_z or A
            K_sub = local_stiffness_3d_timoshenko(E, G, A, Iy, Iz, J, dx,
                                                  shear_area_y=sfy,
                                                  shear_area_z=sfz)
        K_equiv += K_sub

    return K_equiv


def assemble_linear_system(nodes, elements, materials, sections, nprism_segments=1):
    """
    Build global linear stiffness ignoring geometry changes.
    """
    n_nodes = len(nodes)
    ndof = n_nodes * 6
    K_global = np.zeros((ndof, ndof))
    elem_dof_maps = []

    for elem in elements:
        mat = materials[elem.mat_id]
        sec = sections[elem.sec_id]
        i_node = elem.ni - 1
        j_node = elem.nj - 1
        coord_i = nodes[i_node]
        coord_j = nodes[j_node]

        T, L = build_transformation_matrix_3d(coord_i, coord_j)

        if sec.is_prismatic and nprism_segments==1:
            # single-step
            E=mat.E; G=mat.G
            A = sec.A_func(0.0)
            Iy= sec.Iy_func(0.0)
            Iz= sec.Iz_func(0.0)
            J = sec.J_func(0.0)
            if elem.beam_theory=="EB":
                K_loc = local_stiffness_3d_euler_bernoulli(E, G, A, Iy, Iz, J, L)
            else:
                sfy = sec.shear_factor_y or A
                sfz = sec.shear_factor_z or A
                K_loc = local_stiffness_3d_timoshenko(E, G, A, Iy, Iz, J, L,
                                                      shear_area_y=sfy,
                                                      shear_area_z=sfz)
        else:
            # Subdiv for non-prismatic
            K_loc = integrate_nonprismatic_stiffness(L, nprism_segments, mat, sec, elem.beam_theory)

        K_elem = T.T @ K_loc @ T

        dofs_i = [i_node*6 + d for d in range(6)]
        dofs_j = [j_node*6 + d for d in range(6)]
        dofs_map = dofs_i + dofs_j
        elem_dof_maps.append(dofs_map)

        for rloc in range(12):
            rg = dofs_map[rloc]
            for cloc in range(12):
                cg = dofs_map[cloc]
                K_global[rg, cg] += K_elem[rloc, cloc]

    return K_global, elem_dof_maps


def apply_boundary_conditions(K_global, F_global, restrained_dofs):
    idx = [d-1 for d in restrained_dofs]
    K_red = np.delete(K_global, idx, axis=0)
    K_red = np.delete(K_red, idx, axis=1)
    F_red = np.delete(F_global, idx)
    return K_red, F_red

def solve_linear_system(K_red, F_red):
    return np.linalg.solve(K_red, F_red)

def reconstruct_full_displacements(U_red, ndof, restrained_dofs):
    U_full = np.zeros(ndof)
    free_idx = [i for i in range(ndof) if (i+1) not in restrained_dofs]
    for i_f, i_g in enumerate(free_idx):
        U_full[i_g] = U_red[i_f]
    return U_full

##############################################################################
#      4. Simple Geometric Nonlinear (Newton–Raphson) Skeleton
##############################################################################

def assemble_nonlinear_tangent_and_internal(U, nodes, elements, materials, sections, elem_dof_maps):
    """
    Very simplified approach: update node coords by adding U, reassemble K => R_int=K*U
    """
    new_coords = nodes.copy()
    n = len(nodes)
    for i in range(n):
        ux = U[i*6 + 0]
        uy = U[i*6 + 1]
        uz = U[i*6 + 2]
        new_coords[i,0] += ux
        new_coords[i,1] += uy
        new_coords[i,2] += uz

    K_tan, _ = assemble_linear_system(new_coords, elements, materials, sections)
    R_int = K_tan @ U
    return K_tan, R_int

def solve_nonlinear_newton_raphson(U_init, F_global, nodes, elements, materials, sections,
                                   elem_dof_maps, restrained_dofs,
                                   max_iter=20, tol=1e-5):
    U = U_init.copy()
    ndof = len(U)

    for it in range(max_iter):
        K_tan, R_int = assemble_nonlinear_tangent_and_internal(
            U, nodes, elements, materials, sections, elem_dof_maps
        )
        Res = F_global - R_int
        norm_res = np.linalg.norm(Res)
        if norm_res < tol:
            print(f"Nonlinear: converged at iteration {it}, residual={norm_res:.2e}")
            return U

        bc_idx = [d-1 for d in restrained_dofs]
        K_red = np.delete(K_tan, bc_idx, axis=0)
        K_red = np.delete(K_red, bc_idx, axis=1)
        Res_red = np.delete(Res, bc_idx)

        dU_red = np.linalg.solve(K_red, Res_red)
        free_idx = [i for i in range(ndof) if (i+1) not in restrained_dofs]
        for i_f, gi in enumerate(free_idx):
            U[gi] += dU_red[i_f]

    print("Nonlinear analysis did not converge!")
    return U

##############################################################################
#                           5. Example Usage
##############################################################################

def cfunc(val):
    """Helper to return a constant function for cross-section."""
    return lambda x: val

def main_example():
    print("=== Building Example Model ===")
    model = Model3DFrame(name="DemoFrame")

    # 1) Add nodes
    n1 = model.add_node(0,0,0)
    n2 = model.add_node(5,0,0)
    n3 = model.add_node(10,0,0)

    # 2) Add materials
    model.add_material(mat_id=1, E=210e9, G=81e9, name="Steel")

    # 3) Add sections
    model.add_section(
        sec_id=1,
        A_func=cfunc(0.02),
        Iy_func=cfunc(8e-5),
        Iz_func=cfunc(4e-5),
        J_func=cfunc(2e-5),
        is_prismatic=True,
        name="PrismaticSec"
    )
    # Another section, non-prismatic
    model.add_section(
        sec_id=2,
        A_func=lambda x: 0.02 + 0.005*x,
        Iy_func=lambda x: 8e-5 + x*1e-5,
        Iz_func=lambda x: 4e-5 + x*0.5e-5,
        J_func=lambda x: 2e-5 + x*0.5e-5,
        is_prismatic=False,
        name="NonPrismaticSec"
    )

    # 4) Add elements
    model.add_element(n1,n2, mat_id=1, sec_id=1, beam_theory="EB",         name="E1")
    model.add_element(n2,n3, mat_id=1, sec_id=2, beam_theory="Timoshenko", name="E2")

    # 5) Boundary Conditions: fix node1
    model.fix_node(n1)

    # 6) Loads: apply -10kN at node2 in z
    model.initialize_nodal_forces()
    model.apply_nodal_load(node_id=n2, dof_id=3, magnitude=-10000.0)

    # 7) Non-prismatic segments & solver choice
    model.nprism_segments = 3  # subdiv for E2
    model.use_nonlinear = False  # set True for a simple geometric nonlinear run

    # 8) Solve
    U_full, reactions, elem_dof_maps, K_global = model.run_analysis()

    # 9) Optional: Plot the nodes & elements (undeformed)
    if HAS_PLOTLY:
        fig = go.Figure()
        nodes_arr = np.array(model.nodes)
        for e in model.elements:
            i_idx = e.ni - 1
            j_idx = e.nj - 1
            xi, yi, zi = nodes_arr[i_idx]
            xj, yj, zj = nodes_arr[j_idx]
            fig.add_trace(go.Scatter3d(
                x=[xi, xj], y=[yi, yj], z=[zi, zj],
                mode='lines',
                line=dict(color='blue', width=5),
                name=f"{e.name}"
            ))
        for i_node,(xx,yy,zz) in enumerate(model.nodes, start=1):
            fig.add_trace(go.Scatter3d(
                x=[xx], y=[yy], z=[zz],
                mode='markers+text',
                text=[f"N{i_node}"],
                textposition='top center',
                marker=dict(color='red', size=5),
                showlegend=False
            ))
        fig.update_layout(
            title="Enhanced 3D Frame - Example",
            scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'),
        )
        fig.show()
    else:
        print("Plotly not installed. Skipping visualization.")


if __name__=="__main__":
    main_example()


: 